In [5]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from IPython.display import Markdown, display
import logging
import os
import requests

In [7]:
# =========================
# General Config
# =========================
# Set up for bypassing system proxy
os.environ["no_proxy"] = "127.0.0.1,localhost"

# Set up logging for better error handling
DISABLE_LOGGING = False  # Set to True to disable logging, False to enable
logging.basicConfig(
    level=logging.CRITICAL if DISABLE_LOGGING else logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)

In [8]:
# ================================
# Configure Ollama host and port
# ================================
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "127.0.0.1")
OLLAMA_PORT = os.getenv("OLLAMA_PORT", "11434")
OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

In [9]:
# ===========================
# Define the prompt template
# ===========================
code_prompt = PromptTemplate(
    template="Write a very short {language} function that will {task}",
    input_variables=["language", "task"]
)

# =========================
# Input parameters
# =========================
input_params = {"language": "python", "task": "return a list of 5 integers"}

# ================================
# List of models to iterate over
# ================================
models = [
    {"name": "llama3.2:3b", "description": "Llama 3.2"},
    {"name": "gemma3:latest", "description": "Gemma 3 (4.3B)"},
    {"name": "deepseek-r1:1.5b", "description": "DeepSeek-R1 (1.8B)"}
]

In [10]:
# =========================
# Test server connection
# =========================
def test_server_connection(url: str) -> bool:
    try:
        response = requests.get(url, timeout=5)
        logger.info(f"Ollama server status: {response.status_code}")
        return True
    except requests.RequestException as e:
        logger.error(f"❌ Failed to connect to Ollama server at {url}: {e}")
        return False

In [11]:
# # 1) Use Chatgpt api
# api_key = "sk-proj-...."
# llm = OpenAI(
#     openai_api_key=api_key
# )

# 2) Use local LLM
# =================================================
# Function to invoke a model and display the result
# =================================================
def run_model(model: dict, prompt: PromptTemplate, params: dict) -> None:
    try:
        logger.info(f"🚀 Running model: {model['description']} ({model['name']})")
        llm = OllamaLLM(model=model["name"], base_url=OLLAMA_URL, timeout=30)
        code_chain = prompt | llm
        result = code_chain.invoke(params)
        display(Markdown(f"### {model['description']}\n```{params['language']}\n{result}\n```"))
    except Exception as e:
        logger.error(f"⚠️ Error with {model['description']}: {e}")

In [12]:
if not test_server_connection(OLLAMA_URL):
    exit(1)

#Run One Model
run_model(models[0], code_prompt, input_params)

# # Run each model
# for model in models:
#     run_model(model["name"], model["description"], code_prompt, input_params)

17:41:53 - INFO - Ollama server status: 200
17:41:53 - INFO - 🚀 Running model: Llama 3.2 (llama3.2:3b)


17:43:28 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


### Llama 3.2
```python
Here is a simple Python function that returns a list of 5 random integers:

```python
import random

def generate_random_numbers():
    return [random.randint(0, 100) for _ in range(5)]

# Example usage:
print(generate_random_numbers())
```

This will output a list of 5 unique random integers between 0 and 100.
```